Source: https://sec-api.io/resources/fama-french-factor-model
Data F-F_Research_Data_Factors.csv from: https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Research_Data_Factors_CSV.zip

In [50]:
import pandas as pd
import statsmodels.api as sm
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


In [51]:
port_monthly = pd.read_excel('Backtesting Momentum 252 Days Skip 0 Days 21 Days 10 Stocks 2010-01-01 to 2025-03-21 Equal-Weight.xlsx',sheet_name='Performance')
port_monthly = port_monthly[['Date','Total Capital']]
port_monthly = port_monthly.set_index('Date',drop=True)

In [52]:
ff_factors_monthly = pd.read_excel('FF Portfolio Equal-Weight First-Day in Month.xlsx')
ff_factors_monthly = ff_factors_monthly.set_index('Date',drop=True)
# Filter factor dates to match the asset
ff_factors_subset = ff_factors_monthly[ff_factors_monthly.index.isin(port_monthly.index)].copy()
port_monthly= port_monthly[port_monthly.index.isin(ff_factors_subset.index)]
ff_factors_subset = ff_factors_subset[~ff_factors_subset.index.duplicated()]
# Step 3: Calculate excess returns for the asset
port_monthly['Portfolio Return'] = port_monthly['Total Capital'].pct_change()
port_monthly = port_monthly.drop(columns='Total Capital')
port_monthly = port_monthly.iloc[1:]
ff_factors_subset = ff_factors_subset.iloc[1:]
ff_factors_subset["Excess Return"] = port_monthly["Portfolio Return"] - ff_factors_subset["Rf"].astype('float')

In [53]:
# Prepare the independent variables (add a constant to the model)
#X = sm.add_constant(ff_factors_subset[["Mkt - Rf", "SMB", "HML", "MOM"]])
X = sm.add_constant(ff_factors_subset[["MOM"]])
# The dependent variable
y = ff_factors_subset["Excess Return"]
X = X.apply(pd.to_numeric, errors='coerce')  # Convert all columns of X to numeric, turning non-numeric into NaN
y = pd.to_numeric(y, errors='coerce')       # Convert y to numeric
# Run the regression
model = sm.OLS(y, X).fit()
# Display the summary of the regression
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:          Excess Return   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.006
Method:                 Least Squares   F-statistic:                   0.08642
Date:                Sun, 11 May 2025   Prob (F-statistic):              0.769
Time:                        15:38:07   Log-Likelihood:                 233.04
No. Observations:                 152   AIC:                            -462.1
Df Residuals:                     150   BIC:                            -456.0
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0049      0.004     -1.128      0.2